CODICE PULITO

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

: 

In [ ]:
# ✅ Check CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# ✅ Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Paths
BASE_DIR = "/content/drive/MyDrive/proj_cv"
DATASET_PATH = os.path.join(BASE_DIR, "dataset", "paired_dataset_art")
DAMAGED_PATH = os.path.join(DATASET_PATH, "damaged")
UNDAMAGED_PATH = os.path.join(DATASET_PATH, "undamaged")
PROCESSED_PATH = os.path.join(BASE_DIR, "processed_images")

os.makedirs(os.path.join(PROCESSED_PATH, "damaged"), exist_ok=True)
os.makedirs(os.path.join(PROCESSED_PATH, "undamaged"), exist_ok=True)

In [ ]:
# ✅ Preprocessing
IMG_SIZE = (256, 256)

def preprocess_image(path, denoise=True):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    if denoise:
        img = cv2.GaussianBlur(img, (5,5), 0)
    img = img / 255.0
    return img

In [ ]:
# ✅ Process and save images
for img_name in os.listdir(DAMAGED_PATH):
    damaged_path = os.path.join(DAMAGED_PATH, img_name)
    undamaged_name = img_name.replace("-before", "-after")
    undamaged_path = os.path.join(UNDAMAGED_PATH, undamaged_name)

    processed_damaged = preprocess_image(damaged_path)
    cv2.imwrite(os.path.join(PROCESSED_PATH, "damaged", img_name), (processed_damaged * 255).astype(np.uint8))

    if os.path.exists(undamaged_path):
        processed_undamaged = preprocess_image(undamaged_path)
        cv2.imwrite(os.path.join(PROCESSED_PATH, "undamaged", undamaged_name), (processed_undamaged * 255).astype(np.uint8))

In [ ]:
# ✅ Dataset
class ArtDataset(Dataset):
    def __init__(self, damaged_dir, undamaged_dir, transform=None):
        self.samples = []
        self.transform = transform

        damaged_files = os.listdir(damaged_dir)
        undamaged_files = set(os.listdir(undamaged_dir))  # per lookup veloce

        for damaged_file in damaged_files:
            base_name = damaged_file.replace("-before", "-after")
            if base_name in undamaged_files:
                self.samples.append((damaged_file, base_name))

        self.damaged_dir = damaged_dir
        self.undamaged_dir = undamaged_dir

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        damaged_file, undamaged_file = self.samples[idx]

        damaged_img = Image.open(os.path.join(self.damaged_dir, damaged_file)).convert("RGB")
        undamaged_img = Image.open(os.path.join(self.undamaged_dir, undamaged_file)).convert("RGB")

        if self.transform:
            damaged_img = self.transform(damaged_img)
            undamaged_img = self.transform(undamaged_img)

        return damaged_img, undamaged_img


In [ ]:
# ✅ Model: Simple Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(128, 256, 3, 2, 1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, 2, 1, 1), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, 2, 1, 1), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 3, 2, 1, 1), nn.Tanh()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

dataset = ArtDataset(
    damaged_dir=os.path.join(PROCESSED_PATH, "damaged"),
    undamaged_dir=os.path.join(PROCESSED_PATH, "undamaged"),
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
# ✅ Training Loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for damaged, undamaged in dataloader:
        damaged, undamaged = damaged.to(device), undamaged.to(device)
        output = model(damaged)
        loss = criterion(output, undamaged)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as compare_ssim

# ✅ PSNR
def psnr(img1, img2):
    mse = F.mse_loss(img1, img2).item()
    if mse == 0:
        return 100
    return 20 * np.log10(1.0 / np.sqrt(mse))

# ✅ SSIM
def compute_ssim(img1, img2):
    img1 = img1.permute(1, 2, 0).cpu().numpy()
    img2 = img2.permute(1, 2, 0).cpu().numpy()
    img1 = (img1 * 0.5 + 0.5)
    img2 = (img2 * 0.5 + 0.5)
    return compare_ssim(img1, img2, channel_axis=2, data_range=1.0)

# ✅ Valutazione globale
def evaluate_model_metrics(model, dataloader, device):
    model.eval()
    total_psnr = 0.0
    total_ssim = 0.0
    num_samples = 0

    with torch.no_grad():
        for damaged, undamaged in dataloader:
            damaged, undamaged = damaged.to(device), undamaged.to(device)
            restored = model(damaged)

            for i in range(damaged.size(0)):
                p = psnr(restored[i], undamaged[i])
                s = compute_ssim(restored[i], undamaged[i])
                total_psnr += p
                total_ssim += s
                num_samples += 1

    avg_psnr = total_psnr / num_samples
    avg_ssim = total_ssim / num_samples
    print(f"📊 Average PSNR: {avg_psnr:.2f} dB")
    print(f"📊 Average SSIM: {avg_ssim:.4f}")

# ✅ Visualizzazione
def visualize_results(model, dataloader, device, num_images=5):
    model.eval()
    fig, axes = plt.subplots(num_images, 3, figsize=(12, num_images * 4))

    with torch.no_grad():
        for idx, (damaged, undamaged) in enumerate(dataloader):
            damaged, undamaged = damaged.to(device), undamaged.to(device)
            restored = model(damaged)

            for i in range(min(damaged.size(0), num_images)):
                d_img = damaged[i].cpu().permute(1, 2, 0).numpy()
                r_img = restored[i].cpu().permute(1, 2, 0).numpy()
                u_img = undamaged[i].cpu().permute(1, 2, 0).numpy()

                d_img = (d_img * 0.5 + 0.5)
                r_img = (r_img * 0.5 + 0.5)
                u_img = (u_img * 0.5 + 0.5)

                axes[i, 0].imshow(d_img)
                axes[i, 0].set_title("Damaged")
                axes[i, 1].imshow(r_img)
                axes[i, 1].set_title("Restored")
                axes[i, 2].imshow(u_img)
                axes[i, 2].set_title("Ground Truth")
                for ax in axes[i]:
                    ax.axis('off')
            break

    plt.tight_layout()
    plt.show()


In [ ]:
#from eval_and_viz import evaluate_model_metrics, visualize_results

# Valutazione delle metriche
evaluate_model_metrics(model, dataloader, device)

# Visualizzazione risultati
visualize_results(model, dataloader, device, num_images=5)
